In [34]:
%load_ext autoreload
%autoreload 2

# data processing
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

# plotting
import seaborn as sns
import matplotlib.pyplot as plt

# Statistics (correlation)
from scipy.stats import pearsonr, spearmanr

# libraries for the model
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
# custom functions
from pt_helpers import *

## Load the training data
This is the clean data that we processed in the notebook `Training_data_preparation`

In [56]:
data_path = "DATA/training_data/"

df = pd.read_csv(data_path + "VEX_edac_mag_labeled.csv")

In [37]:
df.head()

,DATE_TIME,EDAC,BX,BY,BZ,BT,XSC,YSC,ZSC,RSC,cme
0,2006-04-24 00:00:00,0,12.357658,-1.304164,-3.332425,12.881274,-4451.055178,-3196.485753,-65466.762260,65695.760575,0
1,2006-04-24 00:05:00,0,12.868947,-0.980800,-3.360027,13.340680,-4202.246280,-3138.377907,-65806.350827,66015.078600,0
2,2006-04-24 00:10:00,0,12.857438,-0.871986,-3.487877,13.355384,-3954.000329,-3080.233288,-66137.913808,66327.612616,0
3,2006-04-24 00:15:00,0,12.898635,-0.684986,-2.885689,13.248405,-3705.057257,-3021.761270,-66463.291041,66635.079608,0
4,2006-04-24 00:20:00,0,12.766473,-0.517608,-2.217135,12.972905,-3453.676541,-2962.553108,-66784.717784,66939.596338,0


We separate the feature matrix `X` and the labels `y`.
- `X` will contain X Y and Z coordinates of the magnetic field and the distance from the sun RSC

In [38]:
y = df['cme'].values
X = df[['BX', 'BY', 'BZ', 'RSC']].values

In [39]:
print("Shape of y:", y.shape)
print("Shape of X:", X.shape)

Shape of y: (893069,)
Shape of X: (893069, 4)


## LSTM

In [40]:
time_steps = 36  # 3 hours of data
X_seq, y_seq = create_sequences(X, y, time_steps)

In [41]:
class CMEPredictor(nn.Module):
    def __init__(self, n_features, n_hidden=50, n_layers=2):
        super(CMEPredictor, self).__init__()
        self.lstm = nn.LSTM(input_size=n_features, hidden_size=n_hidden, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(n_hidden, 1)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x[:, -1, :])
        return torch.sigmoid(x)

n_features = X_seq.shape[2]  # Number of features in the dataset
model = CMEPredictor(n_features)

In [42]:
X_train, X_val, y_train, y_val = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, n_features)).reshape(-1, time_steps, n_features)
X_val = scaler.transform(X_val.reshape(-1, n_features)).reshape(-1, time_steps, n_features)

# Convert to PyTorch tensors
train_data = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
val_data = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32))

In [43]:
# Create DataLoaders
batch_size = 32  # Adjust this based on your system's capability
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)

In [44]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
n_epochs = 10

### Training and evaluation

In [45]:
# Training loop with reduced batch size and data loading on-the-fly
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch.view(-1, 1))
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for X_batch, y_batch in val_loader:
            val_output = model(X_batch)
            val_loss += criterion(val_output, y_batch.view(-1, 1)).item()
    val_loss /= len(val_loader)
    
    print(f'Epoch {epoch+1}, Loss: {loss.item()}, Val Loss: {val_loss}')

Epoch 1, Loss: 0.00594579940661788, Val Loss: 0.053774040349407456
Epoch 2, Loss: 0.014139517210423946, Val Loss: 0.047800011793910635
Epoch 3, Loss: 0.11722347140312195, Val Loss: 0.042304934381778625
Epoch 4, Loss: 0.008344695903360844, Val Loss: 0.03521223705175729
Epoch 5, Loss: 0.0020488000009208918, Val Loss: 0.03096859525075252
Epoch 6, Loss: 0.013688743114471436, Val Loss: 0.026611497020232116
Epoch 7, Loss: 0.011936815455555916, Val Loss: 0.02486511793823932
Epoch 8, Loss: 0.0008495384827256203, Val Loss: 0.020216468520680368
Epoch 9, Loss: 0.001588876242749393, Val Loss: 0.016796924547382366
Epoch 10, Loss: 0.0004762269963975996, Val Loss: 0.012527251529922661


In [46]:
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for X_batch, y_batch in val_loader:
        outputs = model(X_batch)
        predicted = (outputs > 0.5).float()  # Using 0.5 as the threshold for binary classification
        predictions.extend(predicted.view(-1).tolist())
        true_labels.extend(y_batch.tolist())

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

Accuracy: 0.9962935383271653
F1 Score: 0.8884770889487871
Precision: 0.9387682449270203
Recall: 0.8433002878157979


## LSTM with class weights

In [47]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# Use the class weights in your loss function
criterion = nn.BCELoss(weight=class_weights[1])  

In [48]:
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch.view(-1, 1))
        loss.backward()
        optimizer.step()

        # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for X_batch, y_batch in val_loader:
            val_output = model(X_batch)
            val_loss += criterion(val_output, y_batch.view(-1, 1)).item()
    val_loss /= len(val_loader)
    
    print(f'Epoch {epoch+1}, Loss: {loss.item()}, Val Loss: {val_loss}')

Epoch 1, Loss: 0.1603228896856308, Val Loss: 0.4320361314307056
Epoch 2, Loss: 0.1396017223596573, Val Loss: 0.399829888507097
Epoch 3, Loss: 0.9034870862960815, Val Loss: 0.24441474721924591
Epoch 4, Loss: 0.005920471623539925, Val Loss: 0.27302157433417157
Epoch 5, Loss: 0.014106993563473225, Val Loss: 0.1628223878501413
Epoch 6, Loss: 0.002864101668819785, Val Loss: 0.1683755014643827
Epoch 7, Loss: 0.03152022138237953, Val Loss: 0.14615272227339263
Epoch 8, Loss: 0.0035317561123520136, Val Loss: 0.16568230441099902
Epoch 9, Loss: 0.0016053959261626005, Val Loss: 0.1535709749060512
Epoch 10, Loss: 0.000489936675876379, Val Loss: 0.13336837656235967


In [49]:
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for X_batch, y_batch in val_loader:
        outputs = model(X_batch)
        predicted = (outputs > 0.5).float()  # Using 0.5 as the threshold for binary classification
        predictions.extend(predicted.view(-1).tolist())
        true_labels.extend(y_batch.tolist())

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

Accuracy: 0.9984491089374996
F1 Score: 0.9549520247194665
Precision: 0.971542025148908
Recall: 0.93891909178126


### Visualization of the results

In [ ]:
fig, ax = plt.subplots(figsize = (20, 6))

dff = df.copy()
dff['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'])
dff = dff.set_index('DATE_TIME')

dff[['BX', 'BY', 'BZ']].plot(ax=ax, alpha=0.25)
#ax.plot(dff.index, dff['BX', 'BY', 'BZ'], color='black', alpha=0.25)
for cme in y_train:
    plt.plot_date(cme)
    #ax.axvline(pd.Timestamp(cme), color='grey')
#y_train.map(lambda cme: ax.axvline(cme, color='red'))
#plt.axvline(y_train[cme])

#plt.xlabel('date')
plt.ylabel('magnetic field strength')
plt.title('Result CME detection')
plt.show()